In [1]:
# CELL 1: SETUP & CONFIGURATION
# ============================================================
import pandas as pd
import numpy as np
import nibabel as nib
from pathlib import Path
from scipy.ndimage import center_of_mass, label
import warnings
warnings.filterwarnings('ignore')

# 1. PATHS
BASE_DIR = Path("/user_data/csimmon2/long_pt")
CSV_FILE = Path('/user_data/csimmon2/git_repos/long_pt/long_pt_sub_info.csv')

# 2. DEFINE THE TWO MAPS
# ------------------------------------------------------------
# Map A: "Standard / Liu" (Broad sensitivity)
# Matches your previous success: Word > Scramble (12), Face > Obj (1)
MAP_A_NAME = "Standard_Liu"
COPE_MAP_A = {
    'face':   1,   # Face > Object
    'house':  2,   # House > Object
    'object': 3,   # Object > Scramble
    'word':   12   # Word > Scramble (The "Broad" one)
}

# Map B: "Differential" (Spatial Stability)
# The one that fixed the drift: Word > Face (10)
MAP_B_NAME = "Differential"
COPE_MAP_B = {
    'face':   1,   # Face > Object
    'house':  2,   # House > Object
    'object': 3,   # Object > Scramble
    'word':   10   # Word > Face (The "Strict" one)
}

# 3. SUBJECT LOADER
def load_subjects():
    df = pd.read_csv(CSV_FILE)
    subjects = {}
    for _, row in df.iterrows():
        sub_id = row['sub']
        if row['patient'] == 1: # Only analyzing patients for now? Or both?
            # Let's grab everyone available
            pass
        
        # Check folders
        if not (BASE_DIR / sub_id).exists(): continue
        
        sessions = [s.name.replace('ses-', '') for s in (BASE_DIR / sub_id).glob('ses-*')]
        sessions = sorted(sessions)
        
        # Filter for Time 1 and Time 2 (assuming longitudinal)
        if len(sessions) < 1: continue
        
        subjects[sub_id] = {
            'group': row['group'],
            'hemi': 'l' if row['intact_hemi'] == 'left' else 'r',
            'sessions': sessions
        }
    return subjects

ANALYSIS_SUBJECTS = load_subjects()
print(f"Loaded {len(ANALYSIS_SUBJECTS)} subjects.")

Loaded 25 subjects.


In [2]:
# CELL 2: ROBUST EXTRACTION FUNCTIONS
# ============================================================

def get_native_file(sub_id, session, file_type, cope_id=None):
    """Robust path finder for native space files"""
    # file_type: 'zstat' or 'cope'
    base = BASE_DIR / sub_id / f'ses-{session}' / 'derivatives' / 'fsl' / 'loc' / 'HighLevel.gfeat'
    
    if cope_id is not None:
        # Handle tuple (10, 1) vs int 10
        cid = cope_id[0] if isinstance(cope_id, tuple) else cope_id
        path = base / f'cope{cid}.feat' / 'stats' / f'{file_type}1.nii.gz'
    else:
        # Generic file (unused here but good for future)
        path = None
        
    return path

def extract_roi_centroid(sub_id, session, cope_id, mask_path):
    """Find peak centroid in native space"""
    zstat_path = get_native_file(sub_id, session, 'zstat', cope_id)
    if not zstat_path or not zstat_path.exists(): return None
    
    # Load Data
    z_img = nib.load(zstat_path)
    z_data = z_img.get_fdata()
    affine = z_img.affine
    
    # Load Mask
    if not mask_path.exists(): return None
    mask_data = nib.load(mask_path).get_fdata() > 0
    
    # Threshold & Mask
    thresh_data = z_data * mask_data
    thresh_data[thresh_data < 2.3] = 0
    
    if np.sum(thresh_data > 0) < 10: return None
    
    # Cluster (Simple largest cluster)
    labeled, n_clust = label(thresh_data > 0)
    if n_clust == 0: return None
    sizes = [np.sum(labeled == i+1) for i in range(n_clust)]
    winner = np.argmax(sizes) + 1
    
    # Centroid of that cluster
    roi_mask = (labeled == winner)
    
    # Weighted Centroid calculation
    coords = np.array(np.where(roi_mask)).T
    weights = thresh_data[roi_mask]
    avg_coord = np.average(coords, axis=0, weights=weights)
    
    # Convert to mm
    centroid_mm = nib.affines.apply_affine(affine, avg_coord)
    return centroid_mm, affine, z_img.shape

def create_sphere_mask(centroid_mm, affine, shape, radius=6):
    """Draw 6mm sphere"""
    # Create grid
    rx, ry, rz = np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2])
    grid = np.array(np.meshgrid(rx, ry, rz, indexing='ij')).reshape(3, -1).T
    
    # To world
    grid_mm = nib.affines.apply_affine(affine, grid)
    
    # Distances
    dists = np.linalg.norm(grid_mm - centroid_mm, axis=1)
    
    # Mask
    mask = np.zeros(shape, dtype=bool)
    mask_indices = grid[dists <= radius].astype(int)
    mask[mask_indices[:,0], mask_indices[:,1], mask_indices[:,2]] = True
    return mask

def run_rsa_pipeline(subjects, cope_map, map_name):
    """Full Pipeline for one Map"""
    print(f"Running Pipeline for: {map_name}")
    results = []
    
    for sub, info in subjects.items():
        hemi = info['hemi']
        
        for cat, cope_id in cope_map.items():
            roi_key = f"{cat}" # Simple name
            
            # 1. Define ROI (Locate it)
            # We look in Session 1 to define the "Search Space" mask usually, 
            # but here we extract PER SESSION (Dynamic) or FIXED? 
            # Let's do DYNAMIC (Session-specific peak) to match your Drift logic.
            
            # We need the search mask for this category
            # Assuming search masks exist in Session 1 folder
            s1 = info['sessions'][0]
            mask_path = BASE_DIR / sub / f'ses-{s1}' / 'ROIs' / f'{hemi}_{cat}_searchmask.nii.gz'
            
            sub_rdms = {} # Store vectors for correlations
            
            for ses in info['sessions']:
                # A. Find Centroid
                res = extract_roi_centroid(sub, ses, cope_id, mask_path)
                if not res: continue
                centroid, affine, shape = res
                
                # B. Draw Sphere
                sphere = create_sphere_mask(centroid, affine, shape)
                
                # C. Extract Betas (For ALL categories in the map)
                betas = []
                cats = []
                
                # We extract betas for every category in the map to build the RDM
                for target_cat, target_cope in cope_map.items():
                    cope_path = get_native_file(sub, ses, 'cope', target_cope)
                    if cope_path and cope_path.exists():
                        val = nib.load(cope_path).get_fdata()[sphere]
                        betas.append(val)
                        cats.append(target_cat)
                
                if not betas: continue
                
                # D. Compute RDM/Distinctiveness
                # Stack
                beta_mat = np.array(betas) # Shape: (n_cats, n_voxels)
                
                # Corr Matrix (Cats x Cats)
                corr_mat = np.corrcoef(beta_mat)
                
                # Distinctiveness for CURRENT Category (cat)
                # Liu Metric: Mean correlation with Non-Preferred
                if cat in cats:
                    idx = cats.index(cat)
                    others = [i for i in range(len(cats)) if i != idx]
                    mean_corr = np.mean(corr_mat[idx, others])
                    
                    results.append({
                        'Map': map_name,
                        'Subject': sub,
                        'Group': info['group'],
                        'ROI': cat,
                        'Session': ses,
                        'Distinctiveness': mean_corr # Lower is better
                    })
    
    return pd.DataFrame(results)

In [3]:
# CELL 3: RUN AND COMPARE
# ============================================================

# 1. Run Map A (Liu)
df_a = run_rsa_pipeline(ANALYSIS_SUBJECTS, COPE_MAP_A, MAP_A_NAME)

# 2. Run Map B (Differential)
df_b = run_rsa_pipeline(ANALYSIS_SUBJECTS, COPE_MAP_B, MAP_B_NAME)

# 3. Combine
if not df_a.empty and not df_b.empty:
    df_all = pd.concat([df_a, df_b])
    
    print("\n" + "="*60)
    print("COMPARING DISTINCTIVENESS (Mean Correlation with Non-Preferred)")
    print("Lower Value = Better Distinctiveness")
    print("="*60)
    
    # Pivot table to see them side-by-side
    summary = df_all.groupby(['Map', 'ROI', 'Group'])['Distinctiveness'].mean().unstack(level=0)
    print(summary)
    
    # Calculate Change (Instability) for Map A
    print("\nCALCULATING LONGITUDINAL INSTABILITY (Map A: Liu)")
    # Filter for paired sessions and calc delta...
    # (Simplified view for quick check)
    
else:
    print("Error: One or both maps failed to extract data.")

Running Pipeline for: Standard_Liu
Running Pipeline for: Differential

COMPARING DISTINCTIVENESS (Mean Correlation with Non-Preferred)
Lower Value = Better Distinctiveness
Map             Differential  Standard_Liu
ROI    Group                              
face   OTC          0.309647      0.129570
       control      0.267109      0.159684
       nonOTC       0.312889      0.125821
house  OTC         -0.134207     -0.167571
       control     -0.038300     -0.056957
       nonOTC      -0.179734     -0.203498
object OTC         -0.173439     -0.165992
       control     -0.021520     -0.151643
       nonOTC      -0.193017     -0.229331
word   OTC          0.393699      0.199683
       control      0.304549      0.171881
       nonOTC       0.360264      0.096173

CALCULATING LONGITUDINAL INSTABILITY (Map A: Liu)


In [4]:
# CELL 4: CALCULATE INSTABILITY (Delta) FOR STANDARD LIU MAP
# ============================================================

print("CALCULATING INSTABILITY (Standard Liu Map)...")

# We use 'df_a' which contains the Standard Liu results
# We need to pivot it to calculate (Ses 2 - Ses 1)

instability_data = []

# Group by Subject and ROI
for (sub, roi), group_data in df_a.groupby(['Subject', 'ROI']):
    # Check if we have both sessions (e.g., '01' and '02' or '1' and '2')
    # We look for the existence of exactly 2 rows usually, or specific session IDs
    
    # Sort by session to be safe
    group_data = group_data.sort_values('Session')
    
    if len(group_data) >= 2:
        # Take the first two sessions found
        val1 = group_data.iloc[0]['Distinctiveness']
        val2 = group_data.iloc[1]['Distinctiveness']
        
        delta = abs(val2 - val1)
        
        cat_type = 'Unilateral' if roi in ['face', 'word'] else 'Bilateral'
        
        instability_data.append({
            'Subject': sub,
            'Group': group_data.iloc[0]['Group'],
            'ROI': roi,
            'Category_Type': cat_type,
            'Delta': delta
        })

df_instability = pd.DataFrame(instability_data)

if not df_instability.empty:
    print("\n" + "="*50)
    print("DISTINCTIVENESS INSTABILITY (Standard Liu Map)")
    print("==================================================")
    
    # 1. Group Summary
    summary = df_instability.groupby(['Group', 'Category_Type'])['Delta'].agg(['mean', 'std', 'count'])
    print(summary)
    
    # 2. OTC Hypothesis Check
    otc = df_instability[df_instability['Group'] == 'OTC']
    if not otc.empty:
        bil = otc[otc['Category_Type'] == 'Bilateral']['Delta'].mean()
        uni = otc[otc['Category_Type'] == 'Unilateral']['Delta'].mean()
        
        print(f"\nOTC Change Pattern:")
        print(f"  Bilateral Change (Obj/House): {bil:.4f}")
        print(f"  Unilateral Change (Face/Word): {uni:.4f}")
        print(f"  Difference: {bil - uni:.4f}")
        
        if bil > uni:
            print("  ✓ RESULT CONFIRMED: Bilateral information is more unstable.")
        else:
            print("  X RESULT DIFFERENT: Unilateral information is more unstable.")
else:
    print("Error: No paired data found.")

CALCULATING INSTABILITY (Standard Liu Map)...

DISTINCTIVENESS INSTABILITY (Standard Liu Map)
                           mean       std  count
Group   Category_Type                           
OTC     Bilateral      0.343449  0.256121     10
        Unilateral     0.166271  0.079572     10
control Bilateral      0.346362  0.246760     14
        Unilateral     0.302994  0.139413     14
nonOTC  Bilateral      0.167545  0.095319     17
        Unilateral     0.163777  0.138069     18

OTC Change Pattern:
  Bilateral Change (Obj/House): 0.3434
  Unilateral Change (Face/Word): 0.1663
  Difference: 0.1772
  ✓ RESULT CONFIRMED: Bilateral information is more unstable.


In [5]:
# CELL 5: LIU vs SCRAMBLE (Head-to-Head)
# ============================================================
import pandas as pd
import numpy as np

# 1. DEFINE THE CONTENDERS
# ------------------------------------------------------------
# Map A: Standard Liu (Mixed)
# Face/House > Object (1, 2), Word/Object > Scramble (12, 3)
MAP_A_NAME = "Standard_Liu"
COPE_MAP_A = {'face': 1, 'house': 2, 'object': 3, 'word': 12}

# Map B: Pure Scramble
# Everything > Scramble (Contrast 10, 11, 3, 12)
# Note: Check your contrast list. Usually:
# Face>Scrm=10, House>Scrm=11, Word>Scrm=12, Obj>Scrm=3
MAP_B_NAME = "Pure_Scramble"
COPE_MAP_B = {'face': 10, 'house': 11, 'object': 3, 'word': 12}

print(f"COMPARING: {MAP_A_NAME} vs {MAP_B_NAME}")

# 2. RUN EXTRACTION (Using your Pipeline Function)
# ------------------------------------------------------------
# Re-using the run_rsa_pipeline function from Cell 2
df_liu = run_rsa_pipeline(ANALYSIS_SUBJECTS, COPE_MAP_A, MAP_A_NAME)
df_scram = run_rsa_pipeline(ANALYSIS_SUBJECTS, COPE_MAP_B, MAP_B_NAME)

# 3. COMBINE & COMPARE
# ------------------------------------------------------------
if not df_liu.empty and not df_scram.empty:
    df_compare = pd.concat([df_liu, df_scram])
    
    print("\n" + "="*60)
    print("DISTINCTIVENESS COMPARISON (Liu vs Scramble)")
    print("Lower Value = Better Distinctiveness")
    print("="*60)
    
    # Pivot for easy reading
    # We want to see how the SAME ROI changes score between maps
    summary = df_compare.groupby(['Map', 'ROI', 'Group'])['Distinctiveness'].mean().unstack(level=0)
    print(summary)
    
    # 4. CALCULATE INSTABILITY FOR SCRAMBLE
    # (We already saw Liu Instability was great. Does Scramble match it?)
    print("\nCALCULATING INSTABILITY (Pure Scramble Map)...")
    
    instability_scram = []
    for (sub, roi), group_data in df_scram.groupby(['Subject', 'ROI']):
        group_data = group_data.sort_values('Session')
        if len(group_data) >= 2:
            val1 = group_data.iloc[0]['Distinctiveness']
            val2 = group_data.iloc[1]['Distinctiveness']
            delta = abs(val2 - val1)
            
            cat_type = 'Unilateral' if roi in ['face', 'word'] else 'Bilateral'
            instability_scram.append({
                'Group': group_data.iloc[0]['Group'],
                'Category_Type': cat_type,
                'Delta': delta
            })
            
    df_inst_scram = pd.DataFrame(instability_scram)
    if not df_inst_scram.empty:
        print("\nSCRAMBLE MAP INSTABILITY (Delta):")
        summary_inst = df_inst_scram.groupby(['Group', 'Category_Type'])['Delta'].agg(['mean', 'count'])
        print(summary_inst)
        
        # OTC Check
        otc = df_inst_scram[df_inst_scram['Group'] == 'OTC']
        if not otc.empty:
            bil = otc[otc['Category_Type'] == 'Bilateral']['Delta'].mean()
            uni = otc[otc['Category_Type'] == 'Unilateral']['Delta'].mean()
            print(f"\nOTC Diff (Bilateral - Unilateral): {bil - uni:.4f}")
            if bil > uni: print("✓ Scramble Map ALSO confirms the hypothesis.")
            else: print("X Scramble Map FAILS the hypothesis.")
else:
    print("Error: Extraction failed.")

COMPARING: Standard_Liu vs Pure_Scramble
Running Pipeline for: Standard_Liu
Running Pipeline for: Pure_Scramble

DISTINCTIVENESS COMPARISON (Liu vs Scramble)
Lower Value = Better Distinctiveness
Map             Pure_Scramble  Standard_Liu
ROI    Group                               
face   OTC           0.503733      0.129570
       control       0.512394      0.159684
       nonOTC        0.456936      0.125821
house  OTC           0.261123     -0.167571
       control       0.339556     -0.056957
       nonOTC        0.055410     -0.203498
object OTC           0.515668     -0.165992
       control       0.496619     -0.151643
       nonOTC        0.452778     -0.229331
word   OTC           0.438878      0.199683
       control       0.490895      0.171881
       nonOTC        0.376583      0.096173

CALCULATING INSTABILITY (Pure Scramble Map)...

SCRAMBLE MAP INSTABILITY (Delta):
                           mean  count
Group   Category_Type                 
OTC     Bilateral      0.351

In [ ]:
# CELL 5: LIU vs SCRAMBLE (Head-to-Head)
# ============================================================
import pandas as pd
import numpy as np

# 1. DEFINE THE CONTENDERS
# ------------------------------------------------------------
# Map A: Standard Liu (Mixed Baselines)
# This uses the "cleaner" contrasts for Faces/Houses
# Face>Object (1), House>Object (2), Obj>Scramble (3), Word>Scramble (12)
MAP_A_NAME = "Standard_Liu"
COPE_MAP_A = {'face': 1, 'house': 2, 'object': 3, 'word': 12}

# Map B: Pure Scramble (Consistent Baseline)
# This uses > Scramble for EVERYTHING.
# Face>Scramble (10), House>Scramble (11), Obj>Scramble (3), Word>Scramble (12)
# Note: Check your contrast list to confirm IDs.
# Usually: Face>Scrm=10, House>Scrm=11.
MAP_B_NAME = "Pure_Scramble"
COPE_MAP_B = {'face': 10, 'house': 11, 'object': 3, 'word': 12}

print(f"COMPARING: {MAP_A_NAME} vs {MAP_B_NAME}")

# 2. RUN EXTRACTION
# ------------------------------------------------------------
# Re-using the run_rsa_pipeline function from Cell 2
print(f"Running extraction for {MAP_A_NAME}...")
df_liu = run_rsa_pipeline(ANALYSIS_SUBJECTS, COPE_MAP_A, MAP_A_NAME)

print(f"Running extraction for {MAP_B_NAME}...")
df_scram = run_rsa_pipeline(ANALYSIS_SUBJECTS, COPE_MAP_B, MAP_B_NAME)

# 3. COMBINE & COMPARE
# ------------------------------------------------------------
if not df_liu.empty and not df_scram.empty:
    df_compare = pd.concat([df_liu, df_scram])

    print("\n" + "="*60)
    print("DISTINCTIVENESS COMPARISON (Liu vs Scramble)")
    print("Lower Value = Better Distinctiveness")
    print("="*60)

    # Pivot table: Rows=ROI/Group, Cols=Map
    summary = df_compare.groupby(['Group', 'ROI', 'Map'])['Distinctiveness'].mean().unstack(level=2)
    
    # Calculate difference
    summary['Diff (Scram - Liu)'] = summary[MAP_B_NAME] - summary[MAP_A_NAME]
    print(summary)

    print("\n INTERPRETATION GUIDE:")
    print(" * Positive Diff: Scramble map is WORSE (Less distinctive)")
    print(" * Zero/Negative: Scramble map is FINE (Equally/More distinctive)")

    # 4. CHECK HYPOTHESIS ON SCRAMBLE MAP
    # Can the Scramble map also replicate the "Bilateral > Unilateral" instability?
    print("\n" + "="*60)
    print(f"HYPOTHESIS CHECK ON {MAP_B_NAME}")
    print("="*60)

    instability_scram = []
    for (sub, roi), group_data in df_scram.groupby(['Subject', 'ROI']):
        group_data = group_data.sort_values('Session')
        if len(group_data) >= 2:
            val1 = group_data.iloc[0]['Distinctiveness']
            val2 = group_data.iloc[1]['Distinctiveness']
            delta = abs(val2 - val1)
            
            cat_type = 'Unilateral' if roi in ['face', 'word'] else 'Bilateral'
            instability_scram.append({
                'Group': group_data.iloc[0]['Group'],
                'Category_Type': cat_type,
                'Delta': delta
            })
            
    df_inst_scram = pd.DataFrame(instability_scram)
    if not df_inst_scram.empty:
        # OTC Check
        otc = df_inst_scram[df_inst_scram['Group'] == 'OTC']
        if not otc.empty:
            bil = otc[otc['Category_Type'] == 'Bilateral']['Delta'].mean()
            uni = otc[otc['Category_Type'] == 'Unilateral']['Delta'].mean()
            print(f"OTC Instability Pattern:")
            print(f"  Bilateral (Obj/House): {bil:.4f}")
            print(f"  Unilateral (Face/Word): {uni:.4f}")
            print(f"  Difference: {bil - uni:.4f}")
            
            if bil > uni: print("✓ SUCCESS: Scramble Map replicates the main finding.")
            else: print("X FAILURE: Scramble Map washes out the finding.")

else:
    print("Error: Extraction failed for one of the maps.")
    
    

COMPARING: Standard_Liu vs Pure_Scramble
Running extraction for Standard_Liu...
Running Pipeline for: Standard_Liu
Running extraction for Pure_Scramble...
Running Pipeline for: Pure_Scramble

DISTINCTIVENESS COMPARISON (Liu vs Scramble)
Lower Value = Better Distinctiveness
Map             Pure_Scramble  Standard_Liu  Diff (Scram - Liu)
Group   ROI                                                    
OTC     face         0.503733      0.129570            0.374163
        house        0.261123     -0.167571            0.428695
        object       0.515668     -0.165992            0.681660
        word         0.438878      0.199683            0.239196
control face         0.512394      0.159684            0.352709
        house        0.339556     -0.056957            0.396513
        object       0.496619     -0.151643            0.648262
        word         0.490895      0.171881            0.319014
nonOTC  face         0.456936      0.125821            0.331115
        house        0

In [7]:
# CELL 6: THE FINAL ROBUSTNESS CHECK (Three MVPA Approaches)
# ============================================================
import pandas as pd
import numpy as np

# 1. DEFINE THE THREE MVPA APPROACHES
# ------------------------------------------------------------
# Method 1: Standard Liu (Texture)
# Maximizes pattern information by removing shared object features
MAP_1_NAME = "Method_1_Liu"
COPE_MAP_1 = {'face': 1, 'house': 2, 'object': 3, 'word': 12}

# Method 2: Pure Scramble (Anchor)
# Maximizes signal strength (Top 10% approach)
MAP_2_NAME = "Method_2_Scramble"
COPE_MAP_2 = {'face': 10, 'house': 11, 'object': 3, 'word': 12}

# Method 3: Differential (Strict)
# Maximizes specificity (e.g., Word > Face)
# Note: Ensure these IDs match your 'Differential' map from earlier
MAP_3_NAME = "Method_3_Differential"
COPE_MAP_3 = {'face': 1, 'house': 2, 'object': 3, 'word': 10} 

print("RUNNING 3-WAY ROBUSTNESS CHECK...")

# 2. RUN PIPELINES
# ------------------------------------------------------------
# We re-run extraction to ensure clean, comparable data
df_1 = run_rsa_pipeline(ANALYSIS_SUBJECTS, COPE_MAP_1, MAP_1_NAME)
df_2 = run_rsa_pipeline(ANALYSIS_SUBJECTS, COPE_MAP_2, MAP_2_NAME)
df_3 = run_rsa_pipeline(ANALYSIS_SUBJECTS, COPE_MAP_3, MAP_3_NAME)

# 3. CALCULATE INSTABILITY (DELTA) FOR ALL THREE
# ------------------------------------------------------------
all_instability = []

for df_curr, method_name in [(df_1, "Liu"), (df_2, "Scramble"), (df_3, "Differential")]:
    if df_curr.empty: continue
    
    # Calculate Abs(Time 2 - Time 1) for every ROI
    for (sub, roi), group_data in df_curr.groupby(['Subject', 'ROI']):
        group_data = group_data.sort_values('Session')
        if len(group_data) >= 2:
            val1 = group_data.iloc[0]['Distinctiveness']
            val2 = group_data.iloc[1]['Distinctiveness']
            delta = abs(val2 - val1)
            
            cat_type = 'Unilateral' if roi in ['face', 'word'] else 'Bilateral'
            
            all_instability.append({
                'Method': method_name,
                'Group': group_data.iloc[0]['Group'],
                'Category_Type': cat_type,
                'Delta': delta
            })

# 4. THE GRAND SUMMARY
# ------------------------------------------------------------
df_final = pd.DataFrame(all_instability)

if not df_final.empty:
    print("\n" + "="*60)
    print("REPRESENTATIONAL INSTABILITY ACROSS 3 MVPA METHODS")
    print("Hypothesis: Bilateral Delta > Unilateral Delta")
    print("="*60)
    
    # Filter for OTC Group only (since that's the main question)
    otc_data = df_final[df_final['Group'] == 'OTC']
    
    # Group by Method and Category Type
    summary = otc_data.groupby(['Method', 'Category_Type'])['Delta'].mean().unstack()
    
    # Calculate the "Effect Size" (Difference)
    summary['Diff (Bi - Uni)'] = summary['Bilateral'] - summary['Unilateral']
    
    print(summary)
    print("-" * 60)
    
    # Verdict logic
    failures = summary[summary['Diff (Bi - Uni)'] <= 0]
    if len(failures) == 0:
        print("✓ ROBUST RESULT: Hypothesis confirmed across ALL three methods.")
    else:
        print(f"X MIXED RESULT: Hypothesis failed in {len(failures)} method(s).")
else:
    print("Error: No data extracted.")

RUNNING 3-WAY ROBUSTNESS CHECK...
Running Pipeline for: Method_1_Liu
Running Pipeline for: Method_2_Scramble
Running Pipeline for: Method_3_Differential

REPRESENTATIONAL INSTABILITY ACROSS 3 MVPA METHODS
Hypothesis: Bilateral Delta > Unilateral Delta
Category_Type  Bilateral  Unilateral  Diff (Bi - Uni)
Method                                               
Differential    0.318425    0.081910         0.236515
Liu             0.343449    0.166271         0.177178
Scramble        0.351563    0.191214         0.160350
------------------------------------------------------------
✓ ROBUST RESULT: Hypothesis confirmed across ALL three methods.


In [8]:
# CELL 7: DRIFT ANALYSIS - STANDARD LIU MAP
# ============================================================
import pandas as pd
import numpy as np

print("TESTING SPATIAL DRIFT USING STANDARD LIU MAP...")

# 1. DEFINE THE MAP (Mixed Baselines)
# Face/House use > Object (Subtraction)
# Word/Object use > Scramble (Baseline)
COPE_MAP_LIU = {'face': 1, 'house': 2, 'object': 3, 'word': 12}

# 2. RUN DRIFT EXTRACTION
# Re-using your drift logic but with Liu Contrasts
liu_drift_results = []

print(f"Extracting centroids for {len(ANALYSIS_SUBJECTS)} subjects...")

for sub_id, info in ANALYSIS_SUBJECTS.items():
    hemi = info['hemi']
    
    for cat, cope_id in COPE_MAP_LIU.items():
        # Get Centroids for Session 1 and Session 2
        # We use the robust 'extract_roi_centroid' function from Cell 2
        
        # Note: We need to find the search mask for this category
        s1 = info['sessions'][0]
        mask_path = BASE_DIR / sub_id / f'ses-{s1}' / 'ROIs' / f'{hemi}_{cat}_searchmask.nii.gz'
        
        # Get Ses 1 Centroid
        res1 = extract_roi_centroid(sub_id, info['sessions'][0], cope_id, mask_path)
        
        # Get Ses 2 Centroid (assuming 2 sessions exist)
        if len(info['sessions']) > 1:
            res2 = extract_roi_centroid(sub_id, info['sessions'][1], cope_id, mask_path)
        else:
            res2 = None
            
        if res1 and res2:
            c1 = res1[0] # The coordinate
            c2 = res2[0]
            
            # Calculate Euclidean Distance (Drift)
            dist = np.linalg.norm(c1 - c2)
            
            cat_type = 'Unilateral' if cat in ['face', 'word'] else 'Bilateral'
            
            liu_drift_results.append({
                'Subject': sub_id,
                'Group': info['group'],
                'ROI': cat,
                'Category_Type': cat_type,
                'Drift_mm': dist
            })

# 3. ANALYZE RESULTS
df_liu_drift = pd.DataFrame(liu_drift_results)

if not df_liu_drift.empty:
    print("\n" + "="*60)
    print("SPATIAL DRIFT (Standard Liu Map)")
    print("Hypothesis: Bilateral Drift > Unilateral Drift")
    print("="*60)
    
    # OTC Summary
    otc = df_liu_drift[df_liu_drift['Group'] == 'OTC']
    if not otc.empty:
        summary = otc.groupby('Category_Type')['Drift_mm'].agg(['mean', 'std', 'count'])
        print(summary)
        
        bil = summary.loc['Bilateral', 'mean']
        uni = summary.loc['Unilateral', 'mean']
        diff = bil - uni
        
        print(f"\nOTC Drift Pattern:")
        print(f"  Bilateral:  {bil:.2f} mm")
        print(f"  Unilateral: {uni:.2f} mm")
        print(f"  Difference: {diff:.2f} mm")
        
        if diff > 0.5: # Threshold for "meaningful" difference in fMRI
            print("\n✓ SUCCESS: Liu Map captures the Drift pattern!")
            print("  (You can use Liu for EVERYTHING)")
        elif diff > 0:
            print("\n~ WEAK SUCCESS: Pattern exists but is small.")
        else:
            print("\nX FAILURE: Liu Map obscures the Drift pattern.")
            print("  (This confirms 'Face > Object' is too noisy for localization)")
else:
    print("Error: No paired data extracted.")

TESTING SPATIAL DRIFT USING STANDARD LIU MAP...
Extracting centroids for 25 subjects...

SPATIAL DRIFT (Standard Liu Map)
Hypothesis: Bilateral Drift > Unilateral Drift
                    mean        std  count
Category_Type                             
Bilateral      27.608486  24.828296     10
Unilateral     17.902994  10.844857      9

OTC Drift Pattern:
  Bilateral:  27.61 mm
  Unilateral: 17.90 mm
  Difference: 9.71 mm

✓ SUCCESS: Liu Map captures the Drift pattern!
  (You can use Liu for EVERYTHING)


In [10]:
# CELL 8: UNIFIED MVPA PIPELINE (Liu 2025) - FIXED NaNs
# ============================================================
import pandas as pd
import numpy as np
from scipy.spatial import procrustes
from scipy.spatial.distance import squareform

# 1. SETUP
# We use Standard Liu because it creates the cleanest 'Distinctiveness' scores
COPE_MAP = {'face': 1, 'house': 2, 'object': 3, 'word': 12}
metrics_data = []

print("RUNNING FINAL STABILITY ANALYSIS (Distinctiveness, Geometry, Procrustes)...")

for sub_id, info in ANALYSIS_SUBJECTS.items():
    if len(info['sessions']) < 2: continue
    
    s1, s2 = info['sessions'][0], info['sessions'][1]
    hemi = info['hemi']
    group = info['group']

    for roi_cat, roi_cope in COPE_MAP.items():
        # A. Define ROI (Using Session 1 Search Mask & Liu Contrast)
        mask_path = BASE_DIR / sub_id / f'ses-{s1}' / 'ROIs' / f'{hemi}_{roi_cat}_searchmask.nii.gz'
        
        # We use the Liu contrast to define the peak (since we proved it works)
        res = extract_roi_centroid(sub_id, s1, roi_cope, mask_path)
        if not res: continue
        centroid, affine, shape = res
        sphere = create_sphere_mask(centroid, affine, shape)
        
        # B. Extract Beta Patterns
        betas_s1, betas_s2 = [], []
        valid = True
        
        for target_cat, target_cope in COPE_MAP.items():
            # Session 1
            p1 = get_native_file(sub_id, s1, 'cope', target_cope)
            if not p1 or not p1.exists(): valid=False; break
            d1 = nib.load(p1).get_fdata()[sphere]
            # Handle zeros/NaNs in raw data
            d1 = np.nan_to_num(d1)
            betas_s1.append(d1)
            
            # Session 2
            p2 = get_native_file(sub_id, s2, 'cope', target_cope)
            if not p2 or not p2.exists(): valid=False; break
            d2 = nib.load(p2).get_fdata()[sphere]
            d2 = np.nan_to_num(d2)
            betas_s2.append(d2)
            
        if not valid: continue
        
        # C. Compute RDMs
        b1 = np.array(betas_s1) # Shape (4, n_voxels)
        b2 = np.array(betas_s2)
        
        # Calculate Correlation Matrix (1 - Corr = RDM)
        # Handle potential division by zero if variance is 0
        with np.errstate(divide='ignore', invalid='ignore'):
            c1 = np.corrcoef(b1)
            c2 = np.corrcoef(b2)
        
        # Clean NaNs immediately
        c1 = np.nan_to_num(c1)
        c2 = np.nan_to_num(c2)
        
        rdm1 = 1 - c1
        rdm2 = 1 - c2
        
        # D. METRIC 1: DISTINCTIVENESS CHANGE
        # (How much did the "Purity" change?)
        idx = list(COPE_MAP.keys()).index(roi_cat)
        others = [i for i in range(4) if i != idx]
        
        dist1 = np.mean(c1[idx, others])
        dist2 = np.mean(c2[idx, others])
        delta_dist = abs(dist2 - dist1)
        
        # E. METRIC 2: GEOMETRY INSTABILITY
        # (Correlation between S1 RDM and S2 RDM)
        v1 = squareform(rdm1, checks=False)
        v2 = squareform(rdm2, checks=False)
        geo_corr = np.corrcoef(v1, v2)[0,1]
        geo_instability = 1 - geo_corr # Higher = More Unstable
        
        # F. METRIC 3: PROCRUSTES ERROR
        # (Warping required to align S1 to S2)
        # Procrustes expects shapes (M, N). We use RDMs directly (4x4) or coordinates.
        # Using RDM rows as coordinates (4 points in 4D space)
        m1, m2, disparity = procrustes(rdm1, rdm2)
        
        # Store Results
        cat_type = 'Unilateral' if roi_cat in ['face', 'word'] else 'Bilateral'
        metrics_data.append({
            'Group': group,
            'ROI': roi_cat,
            'Type': cat_type,
            'Delta_Distinctiveness': delta_dist,
            'Geometry_Instability': geo_instability,
            'Procrustes_Error': disparity
        })

# 4. REPORT
df_final = pd.DataFrame(metrics_data)

if not df_final.empty:
    print("\n" + "="*60)
    print("FINAL STABILITY REPORT (Liu Map)")
    print("Hypothesis: Bilateral > Unilateral (Higher values = More Unstable)")
    print("="*60)
    
    # Filter for OTC Group
    otc = df_final[df_final['Group'] == 'OTC']
    if otc.empty:
        print("No OTC data found.")
    else:
        for metric in ['Delta_Distinctiveness', 'Geometry_Instability', 'Procrustes_Error']:
            print(f"\n--- {metric} ---")
            summary = otc.groupby('Type')[metric].agg(['mean', 'std'])
            print(summary)
            
            bil = summary.loc['Bilateral', 'mean']
            uni = summary.loc['Unilateral', 'mean']
            print(f"Diff (Bi - Uni): {bil - uni:.4f}")
            
            if bil > uni:
                print("✓ Hypothesis Confirmed")
            else:
                print("X Hypothesis Failed")
else:
    print("Error: No data extraction occurred.")

RUNNING FINAL STABILITY ANALYSIS (Distinctiveness, Geometry, Procrustes)...


ValueError: Input matrices must contain >1 unique points